## Análise Exploratória

### Importando pacotes necessários e Extraindo conjunto de dados

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing

# Machine Learning
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier

# Data Visualization
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

'''import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
'''
# Any results you write to the current directory are saved as output.

In [ ]:
# sinasc_df = pd.read_csv('../input/sinasc2016/SINASC_2016_RJ_lab.csv')
# sinasc_df = pd.read_csv('SINASC_2016_RJ_lab.csv')

In [ ]:
sinasc_df.shape

In [ ]:
sinasc_df.head()

In [ ]:
sinasc_df.tail()

In [ ]:
print(sinasc_df.columns)
print('-----------------------------------------------------------------------------')
print(sinasc_df.dtypes)
print('-----------------------------------------------------------------------------')
print(sinasc_df.dtypes.value_counts())
print('-----------------------------------------------------------------------------')

In [ ]:
sinasc_df.index

In [ ]:
sinasc_df.describe()

In [ ]:
sinasc_df.info()

In [ ]:
'''import pandas_profiling
pandas_profiling.ProfileReport(sinasc_df)'''

Os códigos acima foram utilizados para a apresentação de características do conjunto de dados utilizado no estudo. Por esses comandos, é possível observar a dimensão da amostra, o número de casos, o conjunto de variáveis, sua denominação, tipo e estados de atributos, dentre outras características. Desse modo, essas ações compreendem um suporte para se delinear parte da estratégia de análise. Para uma abordagem ampla foram identificados as features iniciais, bem como os tipos listados de cada feature, além do quantitativo total de cada tipo primitivo, observando sua importância futura.

Para complementação da avaliação inicial do dataset, bem como facilitar a analise sem um elevado número de códigos importamos uma formidável ferramenta que é o Pandas Profile para sintetizar os dados observados anteriormente, além de facilitar a visualização das interações entre variáveis sem adição de outros plots e bibliotecas.

Entre outros pontos de visualização foi observada a quantidade de respostas para cada valor possível, isto é, observando uma maior previsibilidade nas respostas, além de estar cientes de atributos que se diferenciam pouco no modelo. Isso poderia ser feito sem o Pandas Profile, com o código: Ex.: sinasc_df.peso.value.counts( ). Entretanto, o mesmo absorve todas as features e possibilita melhor entendimento, novamente, sem adição de várias linhas de código.


## Análise Visual Geral

A análise visual abrangeu aspectos diversos da distribuição dos dados pelas variáveis. Ao se reservar atenção à variável de desfecho, PESO, pode ser observado que essa é uma pouca das variáveis que performam uma escala ou outras tendências. Em destaque, na análise dos histogramas, pode ser observado que a variável Peso possui uma das melhores distribuições entre as classes, sendo uma métrica interessante para ser estudada.

In [ ]:
sinasc_df.hist(bins=50, figsize=(20,15))
plt.show()

In [ ]:
sns.countplot(data=sinasc_df, x='PARTO');

In [ ]:
sns.countplot(data=sinasc_df, x='PARTO', hue='SEXO');

Os gráficos testados utilizam as principais variáveis para relacionais, que figuram sobre o PESO, possibilitando verificar uma maior correlação sem apresentação de um código especifico para essa medição.

In [ ]:
from pandas.plotting import scatter_matrix
attributes = ["QTDGESTANT","PESO", "GESTACAO" ]
scatter_matrix(sinasc_df[attributes], figsize=(12, 8))

### Análise da variável PESO

In [ ]:
sinasc_df.PESO.mean()

In [ ]:
colunas = ["IDADEMAE","PESO"]
sinasc_df[colunas].describe()

In [ ]:
idade_max = sinasc_df.IDADEMAE.max()
idade_min = sinasc_df.IDADEMAE.min()

peso_max = sinasc_df.PESO.max()
peso_min = sinasc_df.PESO.min()

In [ ]:
sinasc_df[(sinasc_df.PESO == peso_max) | (sinasc_df.PESO == peso_min)]

In [ ]:
sinasc_df.plot(kind="scatter", x="PARTO", y="PESO", alpha=.1)

In [ ]:
plt.scatter(sinasc_df["IDADEMAE"],sinasc_df["PESO"], alpha=0.2,
s=10*sinasc_df["APGAR1"], c=sinasc_df.TPROBSON, cmap='viridis')
plt.xlabel("IDADEMAE")
plt.ylabel("PESO");

In [ ]:
sinasc_df.plot(kind="scatter", x="SEMAGESTAC", y="PESO", alpha=.1)

Com essas avaliações, buscou-se observar a distribuição dos dados e sua relação com valores extremos ou outliers. Nota-se que no conjunto de dados há valores destoantes, que podem ser derivados de fatores como mal preenchimento ou erro nas unidades de medida. Foram observados cerca de 220 mil casos, sendo que o peso médio dos recém-nascidos é de cerca de 3 kg,com valores extremos abaixo de 2 kg e acima de 4,5 kg, aproximadamente, com crianças do sexo masculino apresentando a tendência de terem maior peso que crianças do sexo feminino. As mães possuem média de idade de cerca de 27 anos, com idade mínima de 10 anos e máxima de 99 anos (com grande possibilidade de correspondem a um erro no conjunto de dados). Ademais, há ampla sobreposição para os valores de peso entre os tipos de parto, sendo que há bem menos casos em que o tipo de parto foi ignorado. Parece também não haver uma correlação direta entre a idade das mães e o peso das crianças, tal qual para o número de consultas e o peso, todavia, essa variável de desfecho indica tendências quando o número de semanas de gestação é considerado.

## Pré-processamento

### Tratamento de Missing Values

In [ ]:
cat_null = sinasc_df.isnull().sum()
print(cat_null)

cat_null = cat_null/len(sinasc_df)*100
cat_null = cat_null[cat_null>0]
cat_null.sort_values(inplace=True, ascending=False)
cat_null

O algoritmo acima contém elementos que facilitam na análise de valores faltantes, sendo melhor do que funções e métodos separadamente. Em síntese ele retorna o total de missings de cada feature, bem como o percentual dentro de cada uma. O gráfico possibilita visualizar melhor o percentual de missing values de cada features, a atribuição feita no código é uma linha de “corte” de 50%, contudo visando reservar os atributos que podem conter uma boa correção com o “PESO”, foi aplicado no modelo a seguir um corte de 45%, isto é, exclusão das features que estão sobre este percentual. Ademais, foi programada a substituição dos atributos numéricos pela mediana, medida central que expressa melhor assertividade do que a média, por exemplo, principalmente em casos com maior possibilidade de variação numérica. Dentre os atributos categóricos foi substituído pela moda dos valores existentes.


Isso facilita para interpretação do gráfico abaixo, que acrescente análise visual ao problema dos valores faltantes. Como um gráfico de barras simples o mesmo traça uma linha horizontal no ponto 50% da escala. Além disso o percentual anteriormente apresentado será a base para exclusão de das features com mais de 45% de valores faltantes. 

Sendo os demais valores das features permanecentes substituídos pela média das variáveis numéricas e pela mediana com variáveis categóricas.



In [ ]:
null_cat=sinasc_df.isna().sum()/len(sinasc_df)*100
plt.bar(range(len(null_cat)),null_cat)
plt.xlabel('Features')
plt.ylabel('% of NAN values')
plt.plot([0, 25], [50,50], 'r--', lw=1)
plt.xticks(list(range(len(sinasc_df.columns))),list(sinasc_df.columns.values),rotation='vertical')

In [ ]:
for col in sinasc_df.columns.values:
    if (pd.isna(sinasc_df[col]).sum()) > 0: 
        if pd.isna(sinasc_df[col]).sum() > (45/100 * len(sinasc_df)): 
            print(col,"removido") 
            sinasc_df = sinasc_df.drop([col], axis=1) 
        elif (sinasc_df[col].dtype == 'object'):
            sinasc_df[col] = sinasc_df[col].fillna(sinasc_df[col].mode()[0])        
        else: sinasc_df[col] = sinasc_df[col].fillna(sinasc_df[col].median())

### Feature Selection

O peso é considerado como a variável de desfecho, para a qual, é avaliado o índice de correlação com as demais:

In [ ]:
sinasc_df["PESO"] = sinasc_df["PESO"].astype('category').cat.codes

In [ ]:
sinasc_corr = sinasc_df.corr()
corr_values = sinasc_corr["PESO"].sort_values(ascending=False)
corr_values = abs(corr_values).sort_values(ascending=False)

print("Correlação das features numéricas com o resultado em ordem crescente")
print(abs(corr_values).sort_values(ascending=False))

A partir disso, foram excluídas as variáveis com valores de baixa correlação, a exemplo de código de malformação congênita, série escolar da mãe, idade do pai, data da última menstruação,código do país de residência, trabalho de parto induzido ou não induzido, possibilidade de nascimento assistido, número de filhos mortos, escolaridade da mãe, cesáreo anterior ao trabalho de parto, dentre outras:

São excluídas as variáveis com valores de baixa correlação:

In [ ]:
sinasc_df = sinasc_df.drop(columns=['STTRABPART'], axis=1)
sinasc_df = sinasc_df.drop(columns=['TPNASCASSI'], axis=1)
sinasc_df = sinasc_df.drop(columns=['QTDFILMORT'], axis=1)
sinasc_df = sinasc_df.drop(columns=['ESCMAE'], axis=1)
sinasc_df = sinasc_df.drop(columns=['STCESPARTO'], axis=1)
sinasc_df = sinasc_df.drop(columns=['ESCMAEAGR1'], axis=1)
sinasc_df = sinasc_df.drop(columns=['IDANOMAL'], axis=1)
sinasc_df = sinasc_df.drop(columns=['DTCADASTRO'], axis=1)
sinasc_df = sinasc_df.drop(columns=['HORANASC'], axis=1)
sinasc_df = sinasc_df.drop(columns=['ESCMAE2010'], axis=1)
sinasc_df = sinasc_df.drop(columns=['DTNASCMAE'], axis=1)
sinasc_df = sinasc_df.drop(columns=['DTNASC'], axis=1)
sinasc_df = sinasc_df.drop(columns=['TPMETESTIM'], axis=1)
sinasc_df = sinasc_df.drop(columns=['CODMUNRES'], axis=1)
sinasc_df = sinasc_df.drop(columns=['CODMUNNASC'], axis=1)

### Enriquecimento de variáveis e Planilha de Dados Final

As variáveis categóricas foram transformadas em dummies.

In [ ]:
sinasc_df = pd.get_dummies(sinasc_df)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(sinasc_df.drop('PESO',axis=1),sinasc_df['PESO'],test_size=0.3,random_state=42)

O conjunto preparado para a modelagem apresenta 223.224 registros e 28 variáveis (incluindo a variável PESO). 

Remoção da variável de desfecho e separação aleatória do dataset em conjunto de treino e teste (reservando 70% dos registros)

In [ ]:
print(x_train.shape)
print(x_test.shape)

As features que obtiveram valores próximos de 0 indicam pouca ou nenhuma correlação com a variável de interesse, enquanto valores próximos de 1, mostram uma correlação de importância para a nossa discussão, mostrando que algumas variáveis influenciam mais do que outras quando avaliamos o peso ao nascer.

As features excluídas possuem pouca relação com a variável peso, objeto do estudo, e portanto, foram excluídas do processo.

## Modelagem

### Random Forest

In [ ]:
# Random Forest
random_forest = RandomForestClassifier(n_estimators=6, min_samples_leaf=3, max_features=0.5, n_jobs=-1)
random_forest.fit(x_train, y_train)

random_forest.score(x_train, y_train)
acc_random_forest = round(random_forest.score(x_train, y_train) * 100, 2)
print('Acurácia do modelo Random Forest Classifier:',acc_random_forest,"\n")
y_pred = random_forest.predict(x_test)

### Outros Modelos

In [ ]:
# Decision Tree
decision_tree = DecisionTreeClassifier(max_depth = 3)
decision_tree.fit(x_train, y_train)
y_pred2 = decision_tree.predict(x_test)
acc_decision_tree = round(decision_tree.score(x_train, y_train) * 100, 2)

In [ ]:
# Support Vector Machine
linear_svc = LinearSVC()
linear_svc.fit(x_train, y_train)
y_pred4 = linear_svc.predict(x_test)
acc_linear_svc = round(linear_svc.score(x_train, y_train) * 100, 2)

In [ ]:
# Logistic Regression
logreg = LogisticRegression()
logreg.fit(x_train, y_train)
y_pred5 = logreg.predict(x_test)
acc_log = round(logreg.score(x_train, y_train) * 100, 2)

# Resultados

In [ ]:
# Ranking final do percentual de acurácia dos modelos aplicados
results = pd.DataFrame({
    'Model': ['Random Forest','Decision Tree', 'SVM','Logistic Regression'],
    'Score': [acc_random_forest, acc_decision_tree, acc_linear_svc, acc_log]})
    
result_df = results.sort_values(by='Score', ascending=False)
result_df = result_df.set_index('Score')
result_df.head(9)

O modelo baseado em Random Forest foi aquele que obteve maior acurácia dentre os testado, chegando a 81.19% com 6 árvores e 28 features e um job, sendo uma modelo enxuto, pois em nenhuma outra configuração com mais árvores ele conseguiu ser finalizado, retornando várias vezes ao início com reinicialização do Kernel.

Outros modelos também não atingiram o esperado retornando pouco ou quase nada em aspectos de acurácia na predição, sendo incluídos para fins comparativos, conforme a tabela do último código.